## Model having 7 features and 2 outputs 

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn 
from keras.layers import Dense
from keras.models import Sequential
import os

In [45]:
df = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/May6rd5percent_RLV_data 100 points2 .csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
# df3=pd.DataFrame(df.iloc[:,0:3])
input = df.values
output = df.values 
X = input[:,0:7]
y = output[:,7:9]
# Individual Data
h = X[:,0]
v = X[:,1]
s = X[:,2]
ω = X[:,3]
γ = X[:,4]
m = X[:,5]
θ = X[:,6]
# t = X[:,3]
T = y[:,0]
b = y[:,1]

In [46]:
df.head(1)

,h,v,s,omega,gamma,m,theta,Thrust,beta,time
0,4871.8,316.67,1.168900e-16,0.342,-1.309,26230,-1.309,483430.0,0.056463,0.0


In [49]:
from sklearn import preprocessing
X_norm = preprocessing.minmax_scale(X)
y_norm = preprocessing.minmax_scale(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)

In [55]:
def create_model(n):
    model = Sequential()
    model.add(Dense(n,input_shape=(7,),kernel_initializer='uniform'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'tanh')) # since tanh has more nonlinearity we add it here, it also gives -ve values so , some layers which are not necessary will lead to 0 in next layer
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(2,kernel_initializer='uniform',activation = 'sigmoid'))
    return model

In [56]:
# for i in range(4,11,1):
i = 8
model = create_model(2**i)
opt = tf.keras.optimizers.Adam(learning_rate = 0.05)
loss = tf.keras.losses.mean_squared_error
def lr_sch(epoch,lr):
    if (epoch>0) & (epoch<200):
      if epoch%50==0: # for every 100 epochs the learning rate varies as metioned. 
        return lr*np.exp(-0.45)
      else:
        return lr
    elif(epoch>200):
        if epoch%40==0:
            return lr*np.exp(-0.225);
        else:
            return lr
    else:
        return lr

def lr_sch2(epoch,lr):
    if (epoch>0) & (epoch<200):
      if epoch%5==0: # for every 100 epochs the learning rate varies as metioned. 
        return round(lr*np.exp(-0.45),7)
      else:
        return round(lr,7)
    elif(epoch>200):
        if epoch%20==0:
            return round(lr*np.exp(-0.125),7)
        else:
            return round(lr,7)
    else:
        return round(lr,7)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_sch2,verbose = 1)

print("The Number of Hidden units used is: ",2**i)
# lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_sch,verbose = 0)


checkpoint_path =  f"/home/saichaitanya/Chaitanya/Trained models /7 input and 2 output features/{2**i}/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# BATCH_SIZE = 128
batchsize = 32
STEPS_PER_EPOCH = X_train.shape[0] / batchsize
save_period = 5

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1,save_freq=int(save_period*STEPS_PER_EPOCH))

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

model.compile(optimizer = opt, loss = loss, metrics = 'accuracy')
# model.load_weights(checkpoint_path)
model.fit(X_train,y_train,epochs = 100,batch_size = batchsize,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler,cp_callback])
model.evaluate(X_test,y_test)

The Number of Hidden units used is:  256

Epoch 1: LearningRateScheduler setting learning rate to 0.05.
Epoch 1/100
5523/5523 [==============================] - 23s 4ms/step - loss: 0.2176 - accuracy: 0.7472 - lr: 0.0500

Epoch 2: LearningRateScheduler setting learning rate to 0.05.
Epoch 2/100
5523/5523 [==============================] - 22s 4ms/step - loss: 0.2176 - accuracy: 0.7473 - lr: 0.0500

Epoch 3: LearningRateScheduler setting learning rate to 0.05.
Epoch 3/100
5523/5523 [==============================] - 23s 4ms/step - loss: 0.2176 - accuracy: 0.7473 - lr: 0.0500

Epoch 4: LearningRateScheduler setting learning rate to 0.05.
Epoch 4/100
5523/5523 [==============================] - 23s 4ms/step - loss: 0.2176 - accuracy: 0.7473 - lr: 0.0500

Epoch 5: LearningRateScheduler setting learning rate to 0.05.
Epoch 5/100
5505/5523 [============================>.] - ETA: 0s - loss: 0.2176 - accuracy: 0.7473
Epoch 5: saving model to /home/saichaitanya/Chaitanya/Trained models /7 input

Epoch 37/100
5523/5523 [==============================] - 22s 4ms/step - loss: 0.2176 - accuracy: 0.7473 - lr: 0.0021

Epoch 38: LearningRateScheduler setting learning rate to 0.0021426.
Epoch 38/100
5523/5523 [==============================] - 22s 4ms/step - loss: 0.2176 - accuracy: 0.7473 - lr: 0.0021

Epoch 39: LearningRateScheduler setting learning rate to 0.0021426.
Epoch 39/100
5523/5523 [==============================] - 22s 4ms/step - loss: 0.2176 - accuracy: 0.7473 - lr: 0.0021

Epoch 40: LearningRateScheduler setting learning rate to 0.0021426.
Epoch 40/100
5479/5523 [============================>.] - ETA: 0s - loss: 0.2176 - accuracy: 0.7474
Epoch 40: saving model to /home/saichaitanya/Chaitanya/Trained models /7 input and 2 output features/256/cp.ckpt
5523/5523 [==============================] - 22s 4ms/step - loss: 0.2176 - accuracy: 0.7473 - lr: 0.0021

Epoch 41: LearningRateScheduler setting learning rate to 0.0013662.
Epoch 41/100
5523/5523 [============================

Epoch 73/100
5523/5523 [==============================] - 23s 4ms/step - loss: 0.2176 - accuracy: 0.7473 - lr: 9.1800e-05

Epoch 74: LearningRateScheduler setting learning rate to 9.18e-05.
Epoch 74/100
5523/5523 [==============================] - 22s 4ms/step - loss: 0.2176 - accuracy: 0.7473 - lr: 9.1800e-05

Epoch 75: LearningRateScheduler setting learning rate to 9.18e-05.
Epoch 75/100
5439/5523 [============================>.] - ETA: 0s - loss: 0.2177 - accuracy: 0.7473
Epoch 75: saving model to /home/saichaitanya/Chaitanya/Trained models /7 input and 2 output features/256/cp.ckpt
5523/5523 [==============================] - 22s 4ms/step - loss: 0.2176 - accuracy: 0.7473 - lr: 9.1800e-05

Epoch 76: LearningRateScheduler setting learning rate to 5.85e-05.
Epoch 76/100
5523/5523 [==============================] - 22s 4ms/step - loss: 0.2176 - accuracy: 0.7473 - lr: 5.8500e-05

Epoch 77: LearningRateScheduler setting learning rate to 5.85e-05.
Epoch 77/100
4733/5523 [================

KeyboardInterrupt: 